In [1]:
import pandas as pd
from utils import *

config = load_config()
PROJECT_PATH = config.project_path
DATA_PATH = PROJECT_PATH.joinpath("data/processed")

first process pittsburgh

In [2]:
df = pd.read_excel(DATA_PATH.joinpath("pittsburg_dataset.xlsx"))
df.drop(columns='Unnamed: 0', inplace=True)

In [3]:
used_pittsburg = pd.read_excel(DATA_PATH.joinpath("50_pittsburgh.xlsx"))

In [4]:
unused_pits = df[~df.reportID.isin(used_pittsburg.reportID)].copy()

We do mimic

In [5]:
# load mimic dataset

df = pd.read_csv("/data/corpora_alpha/MIMIC/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv.gz", compression="gzip")

In [9]:
df.note_id.nunique()

331794

In [7]:
used_mimic = config.representative_notes

/home/wjang/2024_chatbot_noteaid/data/raw/representative_notes_50_new.xlsx


In [13]:
unused_mimic = df[~df.note_id.isin(used_mimic.note_id)].copy()

Summary

In [14]:
unused_mimic.shape

(331744, 8)

In [15]:
unused_pits.shape

(7789, 2)

In [30]:
unused_mimic.to_pickle(DATA_PATH.joinpath("unused_mimic.pkl"))
unused_pits.to_pickle(DATA_PATH.joinpath("unused_pitts.pkl"))

In [32]:
unused_pits.texts[0]

'[ Report de-identified ( Safe-harbor compliant ) by De-ID v.6.22.07.0]\n\n\nFINAL DIAGNOSES AND REASON FOR ADMISSION : 1 .\nAcute sigmoid diverticulitis . 2.\nHematuria . 3.\nCholestatic jaundice . 4 .\nGallstones . 5 .\nChronic kidney disease . 6 .\nStatus post oophorectomy . 7.\nHypertension . 8.\nFibroid uterus .\n\nPROCEDURES DONE ON THE PATIENT : Cystoscopy and MRCP.\n\nHOSPITAL COURSE : The patient came to the hospital with abdominal pain and was treated for diverticulitis .\nThe patient had elevated liver enzymes during her stay in the hospital .\nThe ultrasound showed gallstones .\nThe CT scan of the abdomen showed acute sigmoid diverticulitis , along with gallstones .\nShe was treated with Cipro and Flagyl for diverticulitis .\nShe had a cystoscopy done for hematuria , which revealed normal study .\nThe MRCP revealed normal study .\nHer liver function tests during the discharge were a total bilirubin of 1.15 , direct 0.53 .\nAST 42 , alkaline phosphatase 156 .\nDuring her adm

In [22]:
df = pd.read_pickle(DATA_PATH.joinpath("unused_mimic.pkl"))

In [24]:
df = df.rename(columns={"text": "texts"})

In [25]:
import random
df.sample(10000, random_state=0).to_pickle(DATA_PATH.joinpath("unused_mimic_sample.pkl"))

In [4]:
import pandas as pd
from utils import *

config = load_config()
PROJECT_PATH = config.project_path
DATA_PATH = PROJECT_PATH.joinpath("data/processed")

data = pd.read_pickle(DATA_PATH.joinpath("unused_mimic_sample.pkl"))

In [ ]:
data = data.sample(100)

In [14]:
data.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,texts
329834,19937555-DS-8,19937555,23138501,DS,8,2150-12-01 00:00:00,2150-12-01 07:34:00,\nName: ___ Unit No: __...
223486,16757061-DS-5,16757061,26732232,DS,5,2145-08-13 00:00:00,2145-08-13 19:13:00,\nName: ___ Unit No: ___\...
126429,13826513-DS-16,13826513,23290385,DS,16,2111-08-13 00:00:00,2111-08-13 22:47:00,\nName: ___ Unit No: ___\n ...
135448,14090748-DS-16,14090748,21011199,DS,16,2130-07-11 00:00:00,2130-07-12 16:05:00,\nName: ___ Unit No: ___\n...
76311,12327003-DS-16,12327003,23268045,DS,16,2184-07-20 00:00:00,2184-07-20 14:56:00,\nName: ___ Unit No: _...


In [12]:
from transformers import AutoTokenizer

path = config.model_path("llama3.2-1B")
tokenizer = AutoTokenizer.from_pretrained(path)

In [31]:
counts = []
words = []
for idx, row in data.iterrows() :
    words.append(len(row['texts'].split()))
    length = len(tokenizer.tokenize(row['texts']))
    counts.append(length)
    


In [33]:
import numpy as np

np.array(counts).mean(), np.array(words).mean()

(3173.73, 1637.55)